In [204]:
import cv2
import requests
import base64
import mediapipe as mp
import numpy as np
from mediapipe.python.solutions.pose import PoseLandmark
from mediapipe.python.solutions.drawing_utils import DrawingSpec
from skimage import io
from batch_face import RetinaFace, LandmarkPredictor, draw_landmarks, Timer
import live_pose_estimator
from live_pose_estimator import SixDRep
import time
import torch
# USE Q TO EXIT PROGRAM AND F TO TOGGLE ADVANCED FACE MESH DISPLAY

In [205]:
def get_landmarks(frame, faces):
    ### Predict landmarks from given face co-ordinates ###
    landmarks = predictor(faces, frame, from_fd=True)
    return landmarks

In [206]:
def draw_landmarks_cv(frame, faces, landmarks):
    ### Draw landmarks on faces using CV2 - Possible to draw multiple faces with a For loop, however we are only interested in having one face in the frame ### 
    frame = draw_landmarks(frame, faces[0][0], landmarks[0])
    return frame

In [207]:
def get_head_pose(frame, faces_pose):
    head_poses = head_pose_estimator(faces_pose, frame, input_face_type='tuple', update_dict=True)
    return head_poses

In [208]:
def draw_head_pose_cube_cv(frame, faces, pose):
    head_pose_estimator.plot_pose_cube(frame, faces[0][0], **pose)

In [209]:
def updated_bbox(landmarks):
    ldm_new = landmarks[0]
    (x1, y1), (x2, y2) = ldm_new.min(0), ldm_new.max(0)
    box_new = np.array([x1, y1, x2, y2])
    box_new[:2] -= 10
    box_new[2:] += 10
    faces = [[box_new, None, None]]
    return faces

In [210]:
BUFFER_PERIOD = 100 # Number of Frames to Ignore Before Starting Detection

pose_estimation_state = False  # True when target detected
target_frames_counter = 0
show_advanced_face_mesh_landmarks = False #Flag to show advanced face mesh for end-user to get a more detailed face mesh, purely visual for end user.

mp_draw = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hol = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh

LEFT_IRIS = [474, 475, 476, 477,]
RIGHT_IRIS = [469, 470, 471, 472]

LEFT_EYE = [7,33,246,161,160,159,158,157,173,133,155,154,153,145,144,163]
LEFT_EYE_CONNECTIONS = [(LEFT_EYE[i], LEFT_EYE[i + 1]) for i in range(len(LEFT_EYE) - 1)]

RIGHT_EYE = [362,398,384,385,386,387,388,466,263,249,390,373,374,380,381,382]
RIGHT_EYE_CONNECTIONS = [(RIGHT_EYE[i], RIGHT_EYE[i + 1]) for i in range(len(RIGHT_EYE) - 1)]

FACE_OVAL = [10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176,
             149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109]

# Define Nose and Mouth connections
NOSE = [0, 4, 6]
NOSE_CONNECTIONS = [(NOSE[i], NOSE[i + 1]) for i in range(len(NOSE) - 1)]

MOUTH_OUTER = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291, 375, 321, 405, 314, 17, 84, 181, 91, 146, 61]
MOUTH_OUTER_CONNECTIONS = [(MOUTH_OUTER[i], MOUTH_OUTER[i + 1]) for i in range(len(MOUTH_OUTER) - 1)]

# Define connections based on the order of landmarks in FACE_OVAL
FACE_OVAL_CONNECTIONS = [
    (FACE_OVAL[i], FACE_OVAL[i + 1]) for i in range(len(FACE_OVAL) - 1)
]
# Add a connection between the last and first landmark to close the oval
FACE_OVAL_CONNECTIONS.append((FACE_OVAL[-1], FACE_OVAL[0]))
LEFT_EYE_CONNECTIONS.append((LEFT_EYE[-1], LEFT_EYE[0]))
RIGHT_EYE_CONNECTIONS.append((RIGHT_EYE[-1], RIGHT_EYE[0]))

# Add a scale factor for the iris circles
scale_factor = 0.5

custom_style = mp_drawing_styles.get_default_pose_landmarks_style()
custom_connections = list(mp_hol.POSE_CONNECTIONS)

custom_face_landmark_style = DrawingSpec(color=(0, 0, 255), thickness=1)

hand_connections_style = DrawingSpec(color=(0, 255, 0), thickness=2)

excluded_pose_landmarks = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 17, 18, 19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 32]

holistic = mp_hol.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

for landmark in excluded_pose_landmarks:
    # Change the way the excluded landmarks are drawn
    custom_style[landmark] = DrawingSpec(color=(255, 255, 0), thickness=None)
    # Remove all connections which contain these landmarks
    custom_connections = [
        connection_tuple
        for connection_tuple in custom_connections
        if landmark not in connection_tuple
    ]

# Switch Between Detection On/Off State
def switch_state(image):
    global pose_estimation_state


def draw(frame):
    global pose_estimation_state
    global target_frames_counter
    global key_actions_detected

    new_key_actions_detected = False

    # Detection
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    img_h, img_w = frame.shape[:2]
    results = holistic.process(image)
    face_results = face_mesh.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Pose Estimation State Transitions
    if ((face_results.multi_face_landmarks or results.pose_landmarks) and not pose_estimation_state) or (not (face_results.multi_face_landmarks or results.pose_landmarks) and pose_estimation_state):
        pose_estimation_state = not pose_estimation_state
        target_frames_counter = 0

    # No Need for Processing if No Target Found
    if pose_estimation_state == False:
        return (image)

    # If Target Found, Increment Target Frames Counter
    target_frames_counter = target_frames_counter + 1

    # On New Target, Allow Buffer Period to Settle into Frame
    if target_frames_counter < BUFFER_PERIOD:
        return (image)

    # Drawing Face, Nose and Mouth Connections
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            # Draw Face Landmarks
            for landmark_num in FACE_OVAL:
                landmark = face_landmarks.landmark[landmark_num]
                cv2.circle(image, (int(landmark.x * img_w), int(landmark.y * img_h)), 4, (0, 0, 255), -1)

            # Draw Face Landmark Connections
            for connection in FACE_OVAL_CONNECTIONS:
                start = face_landmarks.landmark[connection[0]]
                end = face_landmarks.landmark[connection[1]]
                cv2.line(image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

            # Draw Noses Landmark Connections
            for connection in NOSE_CONNECTIONS:
                start = face_landmarks.landmark[connection[0]]
                end = face_landmarks.landmark[connection[1]]
                cv2.line(image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

            #  Draw Mouth Landmark Connections
            for connection in MOUTH_OUTER_CONNECTIONS:
                start = face_landmarks.landmark[connection[0]]
                end = face_landmarks.landmark[connection[1]]
                cv2.line(image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

    # Draw Hand Landmarks & Connections
    mp_draw.draw_landmarks(image, results.right_hand_landmarks, mp_hol.HAND_CONNECTIONS, connection_drawing_spec=hand_connections_style)
    mp_draw.draw_landmarks(image, results.left_hand_landmarks, mp_hol.HAND_CONNECTIONS, connection_drawing_spec=hand_connections_style)

    # Draw Upper Body Pose Landmarks & Connections
    if results.pose_landmarks:
        # Draw Upper Body Pose Landmarks
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if landmark.visibility > 0.5:
                if PoseLandmark(idx) not in excluded_pose_landmarks:
                    cv2.circle(image, (int(landmark.x * img_w), int(landmark.y * img_h)), 5, (0, 0, 255), -1)

        # Draw Upper Body Pose Connections
        for connection in custom_connections:
            start = results.pose_landmarks.landmark[connection[0]]
            end = results.pose_landmarks.landmark[connection[1]]
            if (start.visibility > 0.5 and end.visibility > 0.5):
                if (PoseLandmark(connection[0]) not in excluded_pose_landmarks and PoseLandmark(connection[1]) not in excluded_pose_landmarks):
                    cv2.line(image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

    # Draw Irides
    if face_results.multi_face_landmarks:
        mesh_points = np.array([
            np.multiply([p.x, p.y], [img_w, img_h]).astype(int)
            for p in face_results.multi_face_landmarks[0].landmark
        ])

        (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
        (r_cx, r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])

        center_left = np.array([l_cx, l_cy], dtype=np.int32)
        center_right = np.array([r_cx, r_cy], dtype=np.int32)

        cv2.circle(image, center_left, int(l_radius * scale_factor), (0, 255, 0), 1, cv2.LINE_AA)
        cv2.circle(image, center_right, int(r_radius * scale_factor), (0, 255, 0), 1, cv2.LINE_AA)

    return image


In [211]:
def draw(frame):
    global pose_estimation_state
    global target_frames_counter
    global key_actions_detected
    global show_advanced_face_mesh_landmarks  # Use the global flag here

    new_key_actions_detected = False

    # Detection
    # Upload frame to GPU DOES NOT WORK DUE TO INCOMPATABILITY WITH VERSIONS D:
    #gpu_frame = cv2.cuda_GpuMat()
    #gpu_frame.upload(frame)
    #gpu_frame = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2RGB)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    img_h, img_w = frame.shape[:2]
    #image = gpu_frame.download()  # Download image for further processing
    results = holistic.process(image)
    image.flags.writeable = True
    face_results = face_mesh.process(image)

    # Create a blank image with the same dimensions as the input frame
    blank_image = frame #originally np.zeros_like(frame)

    # Pose Estimation State Transitions
    if ((face_results.multi_face_landmarks or results.pose_landmarks) and not pose_estimation_state) or (not (face_results.multi_face_landmarks or results.pose_landmarks) and pose_estimation_state):
        pose_estimation_state = not pose_estimation_state
        target_frames_counter = 0

    # No Need for Processing if No Target Found
    if pose_estimation_state == False:
        return blank_image

    # If Target Found, Increment Target Frames Counter
    target_frames_counter = target_frames_counter + 1

    # On New Target, Allow Buffer Period to Settle into Frame
    if target_frames_counter < BUFFER_PERIOD:
        return blank_image

    # Check if 'f' is pressed to toggle face landmarks, idk if we need this or not, it's just something i added, doesnt really slow down the program
    if cv2.waitKey(10) & 0xFF == ord('f'):
        show_advanced_face_mesh_landmarks = not show_advanced_face_mesh_landmarks  # Toggle the flag
        
    if show_advanced_face_mesh_landmarks and results.face_landmarks:
        mp_draw.draw_landmarks(blank_image, results.face_landmarks, mp_hol.FACEMESH_TESSELATION, 
                                 mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

    # Drawing Face, Nose, and Mouth Connections/Deprecated/Superceded
    #if face_results.multi_face_landmarks:
    #    for face_landmarks in face_results.multi_face_landmarks:
            # Draw Face Landmarks/Deprecated/Superceded
    #        for landmark_num in FACE_OVAL:
    #            landmark = face_landmarks.landmark[landmark_num]
    #            cv2.circle(blank_image, (int(landmark.x * img_w), int(landmark.y * img_h)), 4, (0, 0, 255), -1)

            # Draw Face Landmark Connections/Disabled For Now/Deprecated
            #for connection in FACE_OVAL_CONNECTIONS:
            #    start = face_landmarks.landmark[connection[0]]
            #    end = face_landmarks.landmark[connection[1]]
            #    cv2.line(blank_image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

            # Draw Nose Landmark Connections
     #       for connection in NOSE_CONNECTIONS:
      #          start = face_landmarks.landmark[connection[0]]
       #         end = face_landmarks.landmark[connection[1]]
        #        cv2.line(blank_image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

            # Draw Mouth Landmark Connections
         #   for connection in MOUTH_OUTER_CONNECTIONS:
          #      start = face_landmarks.landmark[connection[0]]
           #     end = face_landmarks.landmark[connection[1]]
            #    cv2.line(blank_image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

            # Draw Eye Landmark Connections
            #for connection in LEFT_EYE_CONNECTIONS:
             #   start = face_landmarks.landmark[connection[0]]
              #  end  = face_landmarks.landmark[connection[1]]
               # cv2.line(blank_image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

            #for connection in RIGHT_EYE_CONNECTIONS:
             #   start = face_landmarks.landmark[connection[0]]
              #  end  = face_landmarks.landmark[connection[1]]
               # cv2.line(blank_image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)
            

    # Draw Hand Landmarks & Connections
    mp_draw.draw_landmarks(blank_image, results.right_hand_landmarks, mp_hol.HAND_CONNECTIONS, connection_drawing_spec=hand_connections_style)
    mp_draw.draw_landmarks(blank_image, results.left_hand_landmarks, mp_hol.HAND_CONNECTIONS, connection_drawing_spec=hand_connections_style)

    # Draw Upper Body Pose Landmarks & Connections
    if results.pose_landmarks:
        # Draw Upper Body Pose Landmarks
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if landmark.visibility > 0.5:
                if PoseLandmark(idx) not in excluded_pose_landmarks:
                    cv2.circle(blank_image, (int(landmark.x * img_w), int(landmark.y * img_h)), 5, (0, 0, 255), -1)

        # Draw Upper Body Pose Connections
        for connection in custom_connections:
            start = results.pose_landmarks.landmark[connection[0]]
            end = results.pose_landmarks.landmark[connection[1]]
            if (start.visibility > 0.5 and end.visibility > 0.5):
                if (PoseLandmark(connection[0]) not in excluded_pose_landmarks and PoseLandmark(connection[1]) not in excluded_pose_landmarks):
                    cv2.line(blank_image, (int(start.x * img_w), int(start.y * img_h)), (int(end.x * img_w), int(end.y * img_h)), (0, 255, 0), 2)

    # Draw Irides/Deprecated/Superceded
    #if face_results.multi_face_landmarks:
     #   mesh_points = np.array([
      #      np.multiply([p.x, p.y], [img_w, img_h]).astype(int)
       #     for p in face_results.multi_face_landmarks[0].landmark
       # ])
#
 #       (l_cx, l_cy), l_radius = cv2.minEnclosingCircle(mesh_points[LEFT_IRIS])
  #      (r_cx, r_cy), r_radius = cv2.minEnclosingCircle(mesh_points[RIGHT_IRIS])
#
   #     center_left = np.array([l_cx, l_cy], dtype=np.int32)
 #       center_right = np.array([r_cx, r_cy], dtype=np.int32)
#
 #       cv2.circle(blank_image, center_left, int(l_radius * scale_factor), (0, 255, 0), 1, cv2.LINE_AA)
  #      cv2.circle(blank_image, center_right, int(r_radius * scale_factor), (0, 255, 0), 1, cv2.LINE_AA)

    return blank_image


In [212]:
import cv2

cap = cv2.VideoCapture(r"C:\Users\scout\Desktop\Sentinel_AI-main\live_detection\IMG_5307.mp4")

detector = RetinaFace(gpu_id=0) #user gpu_id=-1 for Mac to indicate CPU
predictor = LandmarkPredictor(gpu_id=0) #user gpu_id=-1 for Mac to indicate CPU
head_pose_estimator = SixDRep(gpu_id=0) #user gpu_id=-1 for Mac to indicate CPU
detect_time = time.time()
faces = None

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()  # Capture frame and check return value
    loop_time = time.time()
    # Calculate the time difference
    elapsed_time = time.time() - detect_time
    
    if not ret:
        print("Error reading frame. Check camera connection.")
        break  # Exit loop if frame reading fails
     # Check if n seconds has passed: The shorter the elapsed time - the more face detections are done, but also the lower the fps and efficiency
    if faces is None or elapsed_time >= 1:
        faces = detector(frame, cv=True, threshold=0.5)
        detect_time = time.time()
    else:
        ### This is an efficiency method of predicting the face bound-box - especially for live camera. It uses the min and max values from the results of the previous landmark 'predictor' function. Helps increase the fps rate ###
        ### However, it will not detect new faces, or when a face has gone ###
        faces = updated_bbox(landmarks)

    if len(faces) == 0:
        print("NO face is detected!")
        continue
    ### Predict landmarks from face ###
    landmarks = get_landmarks(frame, faces)

    ### Estimate head pose from face ###
    pose = get_head_pose(frame, faces)
    
    ### Draw landmarks (AND/OR) pose cube ###
    frame = draw_landmarks_cv(frame, faces, landmarks)
    draw_head_pose_cube_cv(frame, faces, pose[0])
    # Assuming you have a draw function to process the frame
    #!!!!!!the draw function makes the program pretty slow, not too sure how to fix or what is causing it.!!!!!!
    #processed_frame = np.zeros_like(frame)
    #when removing the line below and running the program, it is fast however only shows luke/nhat's tracking and not parts affinity.
    #not too sure how to integrate better without as much stuttering :( additionally, not too sure how to overlay both parts shown with black image
    processed_frame = draw(frame) #DISABLING THIS/ENABLING LINE ABOVE MAKES THE SCREEN BLACK/HIDES THE VIDEO AND SHOWS FACE/EYE TRACKING HOWEVER ONLY WITH NHAT/LUKE'S CODE
    processed_frame = draw_landmarks_cv(processed_frame, faces, landmarks) #BASICALLY BREAKS ALL THE OTHER AFFINITY PARTS
    draw_head_pose_cube_cv(processed_frame, faces, pose[0])
    
    # Display the processed frame
    cv2.imshow("RECEIVING VIDEO", frame)

    # Exit the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NO face is detected!
